### Sessions and Serializability:

So, what is this error about?<br>

The problem is that we are storing a review object in our session, we are not storing a dictionary, a list, or any other python specific data type, we are storing an object!<br>

Under the hood, django takes what ever we sent to a session and serializes it to a format called JSON which stands for JavaScript-Object-Notation, and is infact a very popular data format in web development.<br>
The problem is that objects can not be serialized to that, because they do not only contain data and have other things like methods which can not be translated to JSON.<br>

To fix this, we should save something else instead of the object in the session!<br>
What should we save instead? the review id.<br>
Fix the code<br>

Your AddFavoriteView should look like the following block:

In [ ]:
class AddFavoriteView(View):
    def post(self, request):
        review_id = request.POST['review_id']
        request.session['Favorite_review'] = review_id
        return HttpResponseRedirect('/reviews/' + review_id)

Save, reload and click on the favorite button, everything should work without any problem.<br>

There is one problem though, how are we able that this review is the favorit review?<br>

To do that, you need to read from sthe session.<br>
Where (in which view) are we going to read from the session?<br>
In the 'ReviewDetailView'. Because it is responsible for that template.<br>
    - Inside the 'ReviewDetailView' class redefine a method called get_context_data<br>
        - This method is responsible for the data which is provided to the template.
    - Instead of returning the get_content_data from the super class, store it in a variable called context.<br>
    - As you already know you can intract with this new variable (object) like a dictionary, so add a new key called 'is_favorite' and set it to True if its id is equal to the request.session['favorite_review']<br>
        - Do you see the problem! we do not have access to the request here! What should we do?<br>
            - Django has forseen this. If you create a variable and pass self.request to it, that variable works like the request object and you can access sessions data with it.
            - It is worth mentioning that you could access the review object itself, by self.object too.
    - Now you can return the context.<br>

Your ReviewDetailView class should look like the following block:

In [ ]:
class ReviewDetailView(DetailView):
    template_name = 'reviews/review-detail.html'
    model = Review
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        loaded_review = self.object
        request = self.request
        context["is_favorite"] = True if request.session['favorite_review']==loaded_review.id else False
        return context

Navigate to your template and make the neccassary cheanges! you could print a paragraph if the shown review is the favorite review of the user.<br>


My review-detail.html template look like the following block:

In [ ]:
{% extends "base.html" %}
{% load static %}

{% block page_title %}Review Detail{% endblock page_title %}

{% block css_files %}
    <link rel="stylesheet" href="{% static "reviews/review-detail.css" %}">
{% endblock css_files %}

{% block content %}
    <div class='main-wrapper'>
        <h1>User: {{ review.user_name }}</h1>
        <h3>Rating: {{ review.rating }}</h3>
        <p>Feedback: {{ review.review_text }}</p>
        {% if is_favorite %}
            <p class='favorite-text'>This review is marked as your favorite review.</p>
        {% else %}
        <form action="/reviews/favorite" method="post">
            {% csrf_token %}
            <input type="hidden" name="review_id" value='{{ review.id }}'>
            <button>Favorite</button>
        </form>
        {% endif %}
    </div>
{% endblock content %}

If you save,reload and click on the button! you will see that it doesn't work!<br>
Why?<br>
It is due to the type of data we are storing in the session.<br>
What did we store in the session? Yes a number.<br>
The problem is that when a data is converted to JSON it will convert to a string! hence our if statement in the view class is comparing an integer to a sting!<br>
That returns False!<br>
You should know how to fix it, just convert the sessiond data to integer and then compare it.<br>

Your ReviewDetailView class should look like the following block:

In [ ]:
class ReviewDetailView(DetailView):
    template_name = 'reviews/review-detail.html'
    model = Review
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        loaded_review = self.object
        request = self.request
        context["is_favorite"] = True if int(request.session['Favorite_review'])==loaded_review.id else False
        return context

Now everything should work just fine.<br>

You may face an error when you are trying to access data stored in a session and it is usualy a key error.<br>
Why does it happen?<br>
Well, when the site is being loaded for the very first time! there wont be any sessions! hence when you are checking it in the view class you will get a key error!<br>

How should we fix it?<br>
See, in django sessions when ever you are not sure if a data is there, you should access it by get method<br>
Navigate to the view class and change request.session['Favorite_review] to request.session.get['Favorite_review]<br>

### End of Chapter 10